### Table of Contents 目录

* [安装数据及建立环境](#chapter1)
* [创建函数](#chapter2)
    * [根据姓+字+朝代（optional）查询人物](#section_2_1)
    * [根据姓+官职（含俗称）+朝代（optional）查询人物](#section_2_2)
* [调用函数](#chapter3)
    * [根据姓+字查询人物](#section_3_1)
    * [根据姓+字+朝代查询人物](#section_3_2)
    * [根据姓+官职（含俗称）查询人物](#section_3_3)
    * [根据姓+官职（含俗称）+朝代查询人物](#section_3_4)

        

<a name="chapter1"></a>
## 安装数据及建立环境 



In [2]:
# Download CBDB sqlite
!git clone https://github.com/cbdb-project/cbdb_sqlite.git

Cloning into 'cbdb_sqlite'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 28 (delta 1), reused 6 (delta 1), pack-reused 22
Unpacking objects: 100% (28/28), done.


In [3]:
# Explode cbdb_20201110.7z
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive('./cbdb_sqlite/CBDB_20210525.7z').extractall(".")

     |████████████████████████████████| 81kB 4.1MB/s 


In [4]:
# Create connection to database
import sqlite3
global CONN
CONN = sqlite3.connect("./CBDB_20210525.db")

In [5]:
!pip install opencc

     |████████████████████████████████| 768kB 9.4MB/s 


<a name="chapter2"></a>
## 创造函数 


<a name="section_2_1"></a>
### 根据姓+字+朝代查询人物（optional）

In [6]:
# Function to search for a person based on surname and alternative name (e.g. 字)
# Dynasty is optional. If not passed in, all results will be returned.
import opencc
s2t_converter = opencc.OpenCC('s2t.json')
def get_person_by_surname_and_altname(surname, altname, dynasty=None):
    surname = s2t_converter.convert(surname)
    altname = s2t_converter.convert(altname)
    if dynasty:
      dynasty = s2t_converter.convert(dynasty)

    cur = CONN.cursor()
    if dynasty:
      sql_statement = "select * from (select * from BIOG_MAIN INNER JOIN ALTNAME_DATA on BIOG_MAIN.c_personid=ALTNAME_DATA.c_personid where ALTNAME_DATA.c_alt_name_chn='" + altname + "' and BIOG_MAIN.c_surname_chn= '" + surname +"') tmp_db LEFT JOIN DYNASTIES on tmp_db.c_dy=DYNASTIES.c_dy where DYNASTIES.c_dynasty_chn='" + dynasty + "';"
    else:
      sql_statement = "select * from BIOG_MAIN INNER JOIN ALTNAME_DATA on BIOG_MAIN.c_personid=ALTNAME_DATA.c_personid where ALTNAME_DATA.c_alt_name_chn='" + altname + "' and BIOG_MAIN.c_surname_chn='" + surname + "';"
    print(sql_statement)
    cur.execute(sql_statement)
    rows = cur.fetchall()
    count = 0
    for row in rows:
      count += 1
      print("Record"+str(count)+": ")
      print(row)

<a name="section_2_2"></a>
### 根据姓+官职+朝代（optional）查询人物

In [ ]:
# Function to search for a person based on surname and office.
# Dynasty is optional. If not passed in, all results will be returned.
import opencc
s2t_converter = opencc.OpenCC('s2t.json')
def get_person_by_surname_and_office(surname, office, dynasty=None):
    surname = s2t_converter.convert(surname)
    office = s2t_converter.convert(office)
    if dynasty:
      dynasty = s2t_converter.convert(dynasty)
    cur = CONN.cursor()
    if dynasty:
      sql_statement = "select * from (select * from (select distinct(c_personid), OFFICE_CODES.c_office_chn_alt, OFFICE_CODES.c_office_chn from POSTED_TO_OFFICE_DATA JOIN OFFICE_CODES on POSTED_TO_OFFICE_DATA.c_office_id=OFFICE_CODES.c_office_id where OFFICE_CODES.c_office_chn_alt like '%" + office + "%' or OFFICE_CODES.c_office_chn like '%" + office + "%') tmp_db LEFT JOIN BIOG_MAIN on BIOG_MAIN.c_personid=tmp_db.c_personid where BIOG_MAIN.c_surname_chn= '" + surname +"') tmp_db2 LEFT JOIN DYNASTIES on tmp_db2.c_dy=DYNASTIES.c_dy where DYNASTIES.c_dynasty_chn='" + dynasty +"';"  
    else:
      sql_statement = "select * from (select distinct(c_personid), OFFICE_CODES.c_office_chn_alt, OFFICE_CODES.c_office_chn from POSTED_TO_OFFICE_DATA JOIN OFFICE_CODES on POSTED_TO_OFFICE_DATA.c_office_id=OFFICE_CODES.c_office_id where OFFICE_CODES.c_office_chn_alt like '%" + office + "%' or OFFICE_CODES.c_office_chn like '%" + office + "%') tmp_db LEFT JOIN BIOG_MAIN on BIOG_MAIN.c_personid=tmp_db.c_personid where BIOG_MAIN.c_surname_chn= '" + surname + "';"
    print(sql_statement)
    cur.execute(sql_statement)
    rows = cur.fetchall()
    count = 0
    for row in rows:
      count += 1
      print("Record"+str(count)+": ")
      print(row)

<a name="chapter3"></a>
## 调用函数 

<a name="section_3_1"></a>
### 根据姓+字查询人物

In [15]:
# Example: Get person by surname + altname
surname = "吕"
zi = "留卿"
get_person_by_surname_and_altname(surname, zi)

select * from BIOG_MAIN INNER JOIN ALTNAME_DATA on BIOG_MAIN.c_personid=ALTNAME_DATA.c_personid where ALTNAME_DATA.c_alt_name_chn='留卿' and BIOG_MAIN.c_surname_chn='呂';
Record1: 
(14990, 20662, 'Lv Shizhang', '呂師張', None, None, 20657, 0, 17545, 1, 0, 0, None, None, 0, None, None, None, 0, None, None, None, None, 1291, 623, 28, '至元二10八年任衢州路同知', None, 0, None, None, 'Lu9', '呂', 'Shizhang', '師張', 18, 0, 'Lu(9) Shizhang [20662] WDY, 1, 424.', 0, 0, None, None, None, None, None, None, None, None, None, None, None, None, 'TTS', '20070312', None, None, 1, 8478, 20662, 'Liuqing', '留卿', 4, 9599, '2590', None, None, 'load', '20110330', None, None)


<a name="section_3_1"></a>
### 根据姓+字+朝代查询人物 

In [ ]:
# Example: Get person by surname + altname + dynasty
surname = "吕"
zi = "留卿"
dynasty = "元"
get_person_by_surname_and_altname(surname, zi, dynasty)

<a name="section_3_3"></a>
### 根据根据姓+官职（含俗称）查询人物

In [ ]:
# Example: Get person by surname + office
surname = "张"
office = "平章"
get_person_by_surname_and_office(surname, office)

<a name="section_3_4"></a>
### 根据姓+官职（含俗称）+朝代查询人物

In [ ]:
# Example: Get person by surname + office + dynasty
surname = "王"
office = "相公"
dynasty = "宋"
get_person_by_surname_and_office(surname, office, dynasty)

In [ ]:
# Example: Get person by surname + office + dynasty
surname = "刘"
office = "运使"
dynasty = "宋"
get_person_by_surname_and_office(surname, office, dynasty)